<a href="https://colab.research.google.com/github/itdusty/blood_cells_classification/blob/custom_dataset/video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data preparations

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install lightning
!pip install pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.0/660.0 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [ ]:
video_transforms = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Normalize((0.45, 0.45, 0.45),(0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=248, max_size=256),
        # CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
dataset_path = "/content/drive/MyDrive/Cells classification/dataset"

In [ ]:
os.chdir(dataset_path)
os.chdir("..")

In [ ]:
dataframe = pd.read_csv("DataFrame.csv", index_col=0)
dataframe = dataframe.sample(frac=1)
ratio = 0.8
train_size = int(dataframe.shape[0] * ratio)
train_data = dataframe[0:train_size]
test_data = dataframe[train_size:]
train_data, test_data

(          files  erythrocytes  lymphocytes  high_erythrocytes  \
 9     traffic10            19           12                  1   
 270  traffic271             6           20                  0   
 888  traffic889             6            1                  0   
 429  traffic430             3           14                  0   
 296  traffic297             9            9                  0   
 ..          ...           ...          ...                ...   
 992  traffic993             1            8                  0   
 821  traffic822             0            1                  0   
 882  traffic883            10            8                  0   
 418  traffic419            14           11                  1   
 0      traffic1             6            9                  0   
 
      high_lymphocytes  blur  noise  
 9                   1     0      0  
 270                 1     0      0  
 888                 0     0      0  
 429                 1     0      0  
 296            

In [ ]:
class CustomDataset(LabeledVideoDataset):
    def __init__(self, dataset_path, dataframe, target_name, transforms, clip_sampler_type='random', clip_duration=1):
      df = dataframe.reset_index()
      paths = []
      for i, file_name in enumerate(df['files']):
          temp_dict = df.iloc[i].to_dict()
          temp_dict['label'] = df[target_name][i]
          temp_dict.pop(target_name)
          temp_dict.pop('files')
          temp_dict.pop('index')
          paths.append((f"{dataset_path}/{file_name}", temp_dict))
      super().__init__(labeled_video_paths=paths,
                       clip_sampler=make_clip_sampler(clip_sampler_type, clip_duration),
                       transform=transforms, decode_audio=False)

In [ ]:
train_dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                              target_name='high_erythrocytes',
                              transforms=video_transforms)

In [ ]:
train_dataset.num_videos

800

### Model

In [ ]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [ ]:
class TestModel(LightningModule):
    def __init__(self, target_label, num_classes=2):
        super(TestModel, self).__init__()
        # model architecture
        self.video_model = torch.hub.load("facebookresearch/pytorchvideo", "efficient_x3d_xs", pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, num_classes)

        self.lr = 1e-3
        self.batch_size = 8
        self.numworkers = 0
        # evaluation metric
        self.metric = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        # loss function
        self.criterion = nn.CrossEntropyLoss()
        # helpers
        self.target_label = target_label
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.testing_step_outputs = []

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}

    def train_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.training_step_outputs.append({'loss': loss, 'metric': metric})
        print({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_train_epoch_end(self):
        outputs = self.training_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.log('train_loss', loss)
        self.log('train_metric', metric)

    def val_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.validation_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        self.testing_step_outputs.append({'label': label, 'pred': out})
        return {'label': label, 'pred': out}

    def on_test_epoch_end(self):
        outputs = self.testing_step_outputs
        label = torch.cat([x['label'] for x in outputs]).cpu().detach().numpy()
        pred = torch.cat([x['pred'].argmax(dim=1) for x in outputs]).cpu().detach().numpy()
        print(classification_report(label, pred))

In [ ]:
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath="checkpoints", filename="file", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

In [ ]:
model = TestModel(num_classes=2, target_label='high_erythrocytes')
seed_everything(0)
trainer = Trainer(max_epochs=1,
                  precision=16,
                  accumulate_grad_batches=2,
                  enable_progress_bar=True,
                  num_sanity_val_steps=0,
                  callbacks=[lr_monitor, checkpoint_callback])

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
INFO:lightning_fabric.utilities.seed:Global seed set to 0
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /content/drive/MyDrive/Cells classification/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type               | Params
---------------------------------------------------
0 | video_model | EfficientX3d       | 3.8 M 
1 | relu        | ReLU               | 0     
2 | linear      | Linear             | 802   
3 | metric      | MulticlassAccuracy | 0     
4 | criterion   | CrossEntropyLoss   | 0     
---------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.180    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

{'loss': tensor(0.5942, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.6544, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.3750, device='cuda:0')}
{'loss': tensor(0.7545, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.5797, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.6727, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.4833, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, device='cuda:0')}
{'loss': tensor(0.8590, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.5000, device='cuda:0')}
{'loss': tensor(0.5689, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.4638, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, d

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytorchvideo/data/frame_video.py", line 243, in fetch_image
    img_str = np.frombuffer(f.read(), np.uint8)
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py", line 43, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 571, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 980, in _run
    results = self._run_stage()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1023, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_ligh

In [ ]:
trainer.validate(model)

In [ ]:
trainer.test(model)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir lightning_logs